In [ ]:
import xarray as xr
import os

import functions.eddy_feedback as ef
import functions.data_wrangling as dw

In [ ]:
path = '/home/links/ct715/data_storage/reanalysis/jra55_daily/'
ds = xr.open_mfdataset(os.path.join(path, 'jra55_uvtw_ubar_ep.nc'))
ds

<xarray.Dataset> Size: 136GB
Dimensions:  (time: 21550, level: 37, lat: 73, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 172kB 1958-01-01T09:00:00 ... 2016-12-31T0...
  * lon      (lon) float64 1kB 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 584B 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * level    (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    u        (time, level, lat, lon) float32 34GB dask.array<chunksize=(21550, 37, 73, 144), meta=np.ndarray>
    v        (time, level, lat, lon) float32 34GB dask.array<chunksize=(21550, 37, 73, 144), meta=np.ndarray>
    t        (time, level, lat, lon) float32 34GB dask.array<chunksize=(21550, 37, 73, 144), meta=np.ndarray>
    omega    (time, level, lat, lon) float32 34GB dask.array<chunksize=(21550, 37, 73, 144), meta=np.ndarray>
    ubar     (time, level, lat) float32 233MB dask.array<chunksize=(21550, 37, 73), meta=np.ndarray>
    ep1      (time, level, lat) float64 466MB dask.array<chunksize=(21550, 37, 73), meta=np.ndarray>
    ep2      (time, level, lat) float64 466MB dask.array<chunksize=(21550, 37, 73), meta=np.ndarray>
    div1     (time, level, lat) float64 466MB dask.array<chunksize=(21550, 37, 73), meta=np.ndarray>
    div2     (time, level, lat) float64 466MB dask.array<chunksize=(21550, 37, 73), meta=np.ndarray>

In [9]:
print(f"NH: {ef.calculate_efp(ds, data_type='reanalysis', which_div1='div1')}")
print(f"SH: {ef.calculate_efp(ds, data_type='reanalysis', which_div1='div1', calc_south_hemis=True)}")

NH: 0.3872
SH: 0.2116


In [ ]:
def plot_reanalysis_correlation(ds, label='DJF', logscale=True, show_rect=True, latitude='NH',
                                top_atmos=10., cut_poles=False, figsize=(6,6),
                                title_name = '\\nabla_{\\phi} F_{\\phi}', take_seasonal=True,
                                which_div1='div1_pr'):
    """"
    Input: DataArrays of ubar and F_\\phi
            - Dims: (time, level, lat)
                - DATASET MUST BE FULL YEAR FOR SEASONAL MEAN 
            - Subsetted to NH or SH
            - Cut off stratosphere >10. hPa
    
    Output: contourf plot matching Fig.6a in Smith et al., 2022 
    """

    ## SET UP TIME

    # If required, check dimensions and variables are labelled correctly
    correct_dims = all(dim_name in ds.dims for dim_name in ['time', 'level', 'lat'])
    if not correct_dims:
        ds = data.check_dimensions(ds)

    # choose hemisphere
    if latitude == 'NH':
        ds = ds.where(ds.lat >= 0., drop=True)
        rect_box = (25., 600.)
        season='djf'
    elif latitude == 'SH':
        ds = ds.where(ds.lat <= 0., drop=True)
        rect_box = (-75., 600.)
        season='jas'

    # separate time into annual means
    # and use .load() to force the calculation now
    if take_seasonal:
        ds = ds.sel(time=slice('1979', '2016'))
        ds = data.seasonal_mean(ds, season=season)


    # set variables
    ubar = ds.ubar
    div1 = ds[which_div1]

    # calculate correlation using built-in Xarray function
    corr = xr.corr(div1, ubar, dim='time')

    # choose top of atmosphere
    corr = corr.where(corr.level >= top_atmos, drop=True)

    if cut_poles:
        corr = corr.where(corr.lat >= -85., drop=True)
        corr = corr.where(corr.lat <= 85., drop=True)

    #------------------------------------------------------------------

    ## PLOTTING TIME

    # Initiate plot
    plt.figure(figsize=figsize)

    # actual plotting
    plt.contourf(corr.lat.values, corr.level.values, corr, cmap='RdBu_r',
                 levels=np.linspace(-0.9,0.9,19), extend='both')
    plt.colorbar(location='bottom', orientation='horizontal', shrink=0.75, label='correlation',
             extend='both', ticks=[-0.6,-0.2,0.2,0.6])

    # axis alterations
    plt.gca().invert_yaxis()
    plt.xlabel('Latitude $(^\\circ N)$')
    if logscale:
        plt.yscale('log')
        plt.ylabel('Log pressure (hPa)')
    else:
        plt.ylabel('Pressure (hPa)')
    # plt.title('$Corr(\\bar{{u}}, {0})$ - {1}'.format(title_name, label))
    plt.title(f'$Corr(\\bar{{u}}, {title_name})$ - {label}')

    # Plot EFP box
    if show_rect:
        rect = patches.Rectangle(rect_box, 50, -400,
                         fill=False, linewidth=2)
        plt.gca().add_patch(rect)

    plt.show()